# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Necessary imports
import pickle
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2
from sklearn.utils import shuffle
from collections import Counter
import math
from tqdm import tnrange
from tqdm import tqdm_notebook as tqdm

%matplotlib inline

# Load the pickled data
training_file = './traffic_signs_data/train.p'
validation_file= './traffic_signs_data/valid.p'
testing_file = './traffic_signs_data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

# Sanity checks
assert(len(X_train) == len(y_train))
assert(len(X_valid) == len(y_valid))
assert(len(X_test) == len(y_test))

# Load in the 'signnames.csv' as a pd.DataFrame
sign_names_df = pd.read_csv('signnames.csv')

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# Number of training examples
n_train = len(X_train)

# Number of validation examples
n_validation = len(X_valid)

# Number of testing examples.
n_test = len(X_test)

# What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# How many unique classes/labels there are in the dataset.
n_classes = sign_names_df.shape[0]

print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization code goes here.
plt.rcParams['image.interpolation'] = 'nearest'

fig, ax = plt.subplots(figsize=(12, 25))
counter = Counter(y_train)
for c in range(n_classes):
    class_idxs = np.where(y_train == c)[0]
    with sns.axes_style('white'):
        plt.subplot(9, 5, c + 1)
        
    plt.imshow(X_train[np.random.choice(class_idxs)])
    plt.tick_params(
        which='both', bottom='off', left='off', top='off', right='off', labelbottom='off', labelleft='off')
    plt.ylabel(sign_names_df.loc[c].SignName, fontsize=200/(np.max([20,len(sign_names_df.loc[c].SignName)])))
    plt.title('n = {}'.format(counter[c]))   

plt.show()

In [ ]:
### 2. Plot Bar charts to see the distribution of the labels across the training/validation/test sets
matplotlib.rcParams['axes.labelsize'] = 20
matplotlib.rcParams['ytick.labelsize'] = 20
matplotlib.rcParams['xtick.labelsize'] = 20
matplotlib.rcParams['font.size'] = 20

def plot_labels_dist(data):
    n_plots = len(data)
    height = 10 * n_plots
    fig, axes = plt.subplots(n_plots, figsize=(30, height))

    # TODO: add alpha for bar chart
    for idx, ele in enumerate(data):    
        labels, title, color = ele
        counter = Counter(labels)
        x_coordinates = np.arange(len(counter))
        frequencies = counter.values()
        mean = np.mean(list(frequencies))
        
        if n_plots == 1:
            ax = axes
        else:  
            ax = axes[idx]
        
        ax.set_title(title)
        ax.bar(x_coordinates, frequencies, align='center', color=color)
        print('The mean # examples for the {} set is: {:.1f}'.format(title, mean))


        ax.xaxis.set_major_locator(plt.FixedLocator(x_coordinates))
        ax.set_xlabel('Label Id')
        ax.set_ylabel('Frequencies')

        # Plotting a dashed line for the mean frequency
        ax.plot([0., 43.], [mean, mean], "k--")

    plt.show()

plot_labels_dist([(y_train, 'Train', 'c'), (y_valid, 'Validation', 'm'), (y_test, 'Test', 'g')])

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
# Peform data augmentation
IMAGE_SIZE = 32

def centrally_scale_images(X_imgs):
    scales = np.linspace(0.6, 1.1, 5)
    # Calculating tf.image.crop_and_resize() params
    boxes = np.zeros([len(scales), 4], dtype=np.float32)
    for index, scale in enumerate(scales):
        x1 = y1 = 0.5 - 0.5 * scale # To scale centrally
        x2 = y2 = 0.5 + 0.5 * scale
        boxes[index] = np.array([y1, x1, y2, x2], dtype=np.float32)

    box_ind = np.zeros(len(scales), dtype=np.int32)
    crop_size = np.array([IMAGE_SIZE, IMAGE_SIZE], dtype=np.int32)

    X_scale_data = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape=[1, IMAGE_SIZE, IMAGE_SIZE, 3])
    tf_img = tf.image.crop_and_resize(X, boxes, box_ind, crop_size)

    with tf.Session() as session:
        for img in X_imgs:
            batch_img = np.expand_dims(img, axis=0)
            scaled_imgs = session.run(tf_img, feed_dict={X: batch_img})
            X_scale_data.extend(scaled_imgs)

        X_scale_data = np.array(X_scale_data, dtype=np.uint8)
        return X_scale_data

def get_translate_params(index): 
    if index == 0: # Translate left 20%
        offset = np.array([0.0, 0.2], dtype=np.float32)
        size = np.array([IMAGE_SIZE, math.ceil(0.8 * IMAGE_SIZE)], dtype=np.float32)
        w_start = 0
        w_end = int(math.ceil(0.8 * IMAGE_SIZE))
        h_start = 0
        h_end = IMAGE_SIZE
    elif index == 1: # Translate right 20%
        offset = np.array([0.0, -0.2], dtype=np.float32)
        size = np.array([IMAGE_SIZE, math.ceil(0.8 * IMAGE_SIZE)], dtype=np.float32)
        w_start = int(math.floor(0.2 * IMAGE_SIZE))
        w_end = IMAGE_SIZE
        h_start = 0
        h_end = IMAGE_SIZE
    elif index == 2: # Translate top 20%
        offset = np.array([0.2, 0.0], dtype=np.float32)
        size = np.array([math.ceil(0.8 * IMAGE_SIZE), IMAGE_SIZE], dtype=np.float32)
        w_start = 0
        w_end = IMAGE_SIZE
        h_start = 0
        h_end = int(math.ceil(0.8 * IMAGE_SIZE))    
    elif index == 3: # Translate bottom 20%
        offset = np.array([-0.2, 0.0], dtype=np.float32)
        size = np.array([math.ceil(0.8 * IMAGE_SIZE), IMAGE_SIZE], dtype=np.float32)
        w_start = 0
        w_end = IMAGE_SIZE
        h_start = int(math.floor(0.2 * IMAGE_SIZE))
        h_end = IMAGE_SIZE 
        
    return offset, size, w_start, w_end, h_start, h_end

def translate_images(X_imgs):
    X_imgs = np.array(X_imgs, dtype=np.float32)
    offsets = np.zeros([len(X_imgs), 2], dtype=np.float32)
    n_translations = 4
    X_translated_data = []

    tf.reset_default_graph()
    with tf.Session() as session:
        for i in range(n_translations):
            X_translated =\
                np.zeros([len(X_imgs), IMAGE_SIZE, IMAGE_SIZE, 3], dtype=np.float32)
                
            base_offset, size, w_start, w_end, h_start, h_end = get_translate_params(i)
            offsets[:,:] = base_offset
            glimpses = tf.image.extract_glimpse(X_imgs, size, offsets)
            glimpses = session.run(glimpses)
            
            X_translated[:, h_start:h_end, w_start:w_end, :] = glimpses
            X_translated_data.extend(X_translated)
     
    X_translated_data = np.array(X_translated_data, dtype=np.uint8)
    return X_translated_data

def finer_rotate_images(X_imgs):
    tf.reset_default_graph()
    
    X_rotate_data = []
    X = tf.placeholder(tf.float32, shape=[None, IMAGE_SIZE, IMAGE_SIZE, 3])
    radian = tf.placeholder(tf.float32, shape=[len(X_imgs)])
    tf_img = tf.contrib.image.rotate(X, radian)
    
    n_rotations = np.linspace(-15, 15, 6)
    with tf.Session() as session:
        for degree in n_rotations:
            radian_val = degree * np.pi / 180
            radian_arr = [radian_val] * len(X_imgs)
            rotated_imgs = session.run(tf_img, feed_dict={
                X: X_imgs, radian: radian_arr
            })
            X_rotate_data.extend(rotated_imgs)

    X_rotate_data = np.array(X_rotate_data, dtype=np.uint8)
    return X_rotate_data

def add_gaussian_noise(X_imgs):
    gaussian_noise_imgs = []
    row, col, _ = X_imgs[0].shape
    
    # Gaussian distribution params
    mean = 0.0; var = 0.1; std = var ** 0.5
    for X_img in X_imgs:
        X_img = X_img.astype(np.float32)
        gaussian = np.random.random((row, col, 1)).astype(np.float32)
        gaussian = np.concatenate((gaussian, gaussian, gaussian), axis = 2)
        gaussian_img = cv2.addWeighted(X_img, 0.75, 0.25 * gaussian, 0.25, 0)
        gaussian_noise_imgs.append(gaussian_img)

    gaussian_noise_imgs = np.array(gaussian_noise_imgs, dtype = np.uint8)

    return gaussian_noise_imgs

def augment_data(X_data, y_data):
    # Get a list of labels that have low samples
    # Get all the images for those labels
    # for each one fo those images choose from 1of4 tranforms
    # extend the training set
    #plot the resulting dist
    # The density param allows us to get values of the pdf at the bin
    augmented_X_data = []
    augmented_y_data = []
    
    hist, _ = np.histogram(y_data, n_classes, density=True)
    labels_to_augment = np.where(hist < 0.02)[0]
    
    for label in labels_to_augment:
        images = X_data[y_data == label]
        gen_images = []
        
        for img in tqdm(images, desc='Label {}'.format(label)):
            aug_type = np.random.randint(0, 3)
            if aug_type == 0:
                gen_images.extend(centrally_scale_images([img]))
            elif aug_type == 1:
                gen_images.extend(translate_images([img]))
            elif aug_type == 2:
                gen_images.extend(finer_rotate_images([img]))
            elif aug_type == 3:
                gen_images.extend(add_gaussian_noise([img]))
        
        print('Generated {} new images for label {}.'.format(len(gen_images), label))
        augmented_X_data.extend(gen_images)
        augmented_y_data.extend([label] * len(gen_images))

    print('Image augmentation process complete.')
    return np.array(augmented_X_data, dtype=np.uint8), np.array(augmented_y_data, dtype=np.uint8)

In [ ]:
# Visualise the different data augmentation techniques on an image from the data set
img = X_train[2238]

gs = gridspec.GridSpec(1, 5)
gs.update(wspace = 0.30, hspace = 2)

fig, ax = plt.subplots(figsize = (16, 16))
plt.subplot(gs[0])
plt.imshow(img)
plt.title('Base Image')
plt.subplot(gs[1])
plt.imshow(centrally_scale_images([img])[0])
plt.title('Central Scaling')
plt.subplot(gs[2])
plt.imshow(translate_images([img])[0])
plt.title('Translation')
plt.subplot(gs[3])
plt.imshow(finer_rotate_images([img])[0])
plt.title('Rotation')
plt.subplot(gs[4])
plt.imshow(add_gaussian_noise([img])[0])
plt.title('Gaussian Noise')
plt.show()

In [ ]:
# Perform data augmentation for the training set
X_train_aug, y_train_aug = augment_data(X_train, y_train)

# Sanity check
assert(len(X_train_aug) == len(y_train_aug))

In [ ]:
### Image augmentation summary
print('Total # images prior to augmentation {}'.format(len(X_train)))
print('Total # of augmented images {}'.format(len(X_train_aug)))

# Combine the augmented data with existing training set
X_train = np.concatenate((X_train, X_train_aug), axis=0)
y_train = np.concatenate((y_train, y_train_aug), axis=0)
print('Total # of images after combination with aug data {}'.format(len(X_train)))

### Visualise the new label distribution
plot_labels_dist([(y_train, 'Train', 'c')])

In [ ]:
# def mean_normalisation(data):
#     image_values = np.arange(0, 256)
#     mu, sigma = np.mean(image_values), np.std(image_values)
#     data = (data - mu) / sigma
#     return data

# def rgb2gray(X):
#     X = 0.2989 * X[...,0] + 0.5870 * X[...,1] + 0.1140 * X[...,2]
#     X = np.sum(X/3, axis=2, keepdims=True)
#     return X.squeeze()
# 
# plt.imshow(rgb2gray(X_train[71030]).squeeze(), cmap='gray')
# a = rgb2gray(mean_normalisation(X_train[25500]))
# a = mean_normalisation(rgb2gray(X_train[23500]))
# a = rgb2gray(X_train[34561] - X_train.mean(axis=0))

# a = mean_normalisation(X_train[2350])
# a.shape
# plt.imshow(a, cmap='gray')
# plt.imshow(X_train[25500])
# X_train[2900]
# plt.imshow(rgb2gray(centrally_scale_images([X_train[2350]])[0]), cmap='gray')

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.

def mean_normalisation(data):
    image_values = np.arange(0, 256)
    mu, sigma = np.mean(image_values), np.std(image_values)
    data = (data - mu) / sigma
    return data    

# Convert the color space for the Training set from RGB -> Gray scale
def rgb2gray(X):
    X = 0.2989 * X[...,0] + 0.5870 * X[...,1] + 0.1140 * X[...,2]
    return X

# Perform normalisation over the training/validation/test set to achieve mu = 0 and sigma = 1
X_train = rgb2gray(mean_normalisation(X_train))
X_valid = rgb2gray(mean_normalisation(X_valid))
X_test  = rgb2gray(mean_normalisation(X_test))

In [ ]:
# Sanity check the shape after color space change from RGB -> Gray scale
print('Shape {} Mean {:.3f} Std {:.3f} Min {:.3f} Max {:.3f}'
      .format(X_train.shape, X_train.mean(), X_train.std(), X_train.min(), X_train.max()))

# Visualise some of the gray scale images
plt.imshow(X_train[2350], cmap='gray')

In [ ]:
### Pickle the pre-processed data sets
print('CHECKPOINT: Pickling the pre-processed data sets...')
try:
    with open('./preprocessed_dataset.p', 'wb') as f:
        d = {
            'features_train': X_train,
            'labels_train': y_train,
            'features_valid': X_valid,
            'labels_valid': y_valid,
            'features_test': X_test,
            'labels_test': y_test,
        }

        pickle.dump(d, f)
except e:
    print('Failed to pickle the data sets! Exception: ' + e)
    
print('Successfully pickled the data sets!')

### Model Architecture

In [1]:
# Necessary imports
import pickle
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2
from sklearn.utils import shuffle
from collections import Counter
import math
from tqdm import tnrange
from tqdm import tqdm_notebook as tqdm

%matplotlib inline

# # Load the pickled data
data_file = './preprocessed_dataset.p'
with open(data_file, mode='rb') as f:
    data = pickle.load(f)
    
X_train, y_train = data['features_train'], data['labels_train']
X_valid, y_valid = data['features_valid'], data['labels_valid']
X_test, y_test = data['features_test'], data['labels_test']

# Sanity checks
assert(len(X_train) == len(y_train))
assert(len(X_valid) == len(y_valid))
assert(len(X_test) == len(y_test))

n_classes = 43

In [2]:
### Define the model hyper params
EPOCHS = 10
BATCH_SIZE = 128
# LEARNING_RATE = 0.005
LEARNING_RATE = 0.001
N_LABELS = n_classes
DROPOUT = 0.8

X = tf.placeholder(tf.float32, shape=[None, 32, 32])
labels = tf.placeholder(tf.int32, shape=[None])
one_hot_labels = tf.one_hot(labels, N_LABELS) # One-hot encode the labels
keep_prob = tf.placeholder(tf.float32)
keep_prob_conv = tf.placeholder(tf.float32)
learning_rate = tf.placeholder(tf.float32)

In [3]:
### Implement the model architecture

def kaiming_he_init(weight_shape, layer_type='fc'):
    fan_in, init_value = None, None
    
    if layer_type == 'conv':
        fan_in = weight_shape[0] * weight_shape[1] * weight_shape[2]
        init_value = tf.truncated_normal(weight_shape) * tf.sqrt(2 / fan_in)
    elif layer_type == 'fc':
        fan_in = weight_shape[0]
        init_value = tf.truncated_normal(weight_shape) * tf.sqrt(2 / fan_in)
    
    return tf.Variable(init_value)

# layer_depth = {
#     'c1': 8,
#     'c2': 8,
#     'c3': 16,
#     'c4': 16,
#     'c5': 32,
#     'c6': 32,
#     'fc1': 128,
# #     'fc2': 64,
#     'out': N_LABELS
# }

# weights = {
#     'c1': kaiming_he_init([5, 5, 1, layer_depth['c1']], 'conv'),
#     'c2': kaiming_he_init([5, 5, layer_depth['c1'], layer_depth['c2']], 'conv'),
#     'c3': kaiming_he_init([3, 3, layer_depth['c2'], layer_depth['c3']], 'conv'),
#     'c4': kaiming_he_init([3, 3, layer_depth['c3'], layer_depth['c4']], 'conv'),
#     'c5': kaiming_he_init([3, 3, layer_depth['c4'], layer_depth['c5']], 'conv'),
#     'c6': kaiming_he_init([3, 3, layer_depth['c5'], layer_depth['c6']], 'conv'),
#     'fc1': kaiming_he_init([4 * 4 *  layer_depth['c6'], layer_depth['fc1']]),
# #     'fc2': kaiming_he_init([layer_depth['fc1'], layer_depth['fc2']]),
#     'out': kaiming_he_init([layer_depth['fc1'], layer_depth['out']])
# }

# biases = {
#     'c1': tf.Variable(tf.truncated_normal([layer_depth['c1']])),
#     'c2': tf.Variable(tf.truncated_normal([layer_depth['c2']])),
#     'c3': tf.Variable(tf.truncated_normal([layer_depth['c3']])),
#     'c4': tf.Variable(tf.truncated_normal([layer_depth['c4']])),
#     'c5': tf.Variable(tf.truncated_normal([layer_depth['c5']])),
#     'c6': tf.Variable(tf.truncated_normal([layer_depth['c6']])),
#     'fc1': tf.Variable(tf.truncated_normal([layer_depth['fc1']])),
# #     'fc2': tf.Variable(tf.truncated_normal([layer_depth['fc2']])),
#     'out': tf.Variable(tf.truncated_normal([layer_depth['out']]))
# }

# def conv2d(input_vol, W, b, stride=1, padding='SAME'):
#     conv_layer = tf.nn.conv2d(input_vol, W, strides=[1, stride, stride, 1], padding=padding)
#     conv_layer = tf.nn.bias_add(conv_layer, b)
#     conv_layer = tf.nn.relu(conv_layer)
#     return conv_layer
    
# def maxpool2d(input_vol, k=2, stride=2):
#     pool = tf.nn.max_pool(input_vol, ksize=[1, k, k, 1], strides=[1, stride, stride, 1], padding='SAME')
#     return pool

# def traffic_sign_cnn(X, weights, biases, keep_prob, keep_prob_conv):
#     # CONV1    Input = [32x32x1] Output = [32x32x16]
#     X = tf.expand_dims(X, -1)
#     conv1 = conv2d(X, weights['c1'], biases['c1'])
    
#     # CONV2    Input = [32x32x16] Output = [32x32x16]
#     # MAXPOOL  Input = [32x32x16] Output = [16x16x16]
#     conv2 = conv2d(conv1, weights['c2'], biases['c2'])
#     conv2 = maxpool2d(conv2)
#     con2 = tf.nn.dropout(conv2, keep_prob=keep_prob_conv)
    
#     # CONV3    Input = [16x16x16] Output = [16x16x32]
#     conv3 = conv2d(conv2, weights['c3'], biases['c3'])

#     # CONV4    Input = [16x16x32] Output = [16x16x32]
#     # MAXPOOL  Input = [16x16x32] Output = [8x8x32]
#     conv4 = conv2d(conv3, weights['c4'], biases['c4'])
#     conv4 = maxpool2d(conv4)
#     con4 = tf.nn.dropout(conv4, keep_prob=keep_prob_conv)
    
#     # CONV5    Input = [8x8x32] Output = [8x8x64]
#     conv5 = conv2d(conv4, weights['c5'], biases['c5'])

#     # CONV6    Input = [8x8x64] Output = [8x8x64]
#     # MAXPOOL  Input = [8x8x64] Output = [4x4x64]
#     conv6 = conv2d(conv5, weights['c6'], biases['c6'])
#     conv6 = maxpool2d(conv6)
#     con6 = tf.nn.dropout(conv6, keep_prob=keep_prob_conv)
    
#     # FC1     Input = [1024] Output =  [256]
#     fc1 = tf.contrib.layers.flatten(conv6)
#     fc1 = tf.matmul(fc1, weights['fc1']) + biases['fc1']
#     fc1 = tf.nn.relu(fc1)
#     fc1 = tf.nn.dropout(fc1, keep_prob=keep_prob)
    
#     # FC2     Input = [256] Output =  [128]
# #     fc2 = tf.matmul(fc1, weights['fc2']) + biases['fc2']
# #     fc2 = tf.nn.relu(fc2)
# #     fc2 = tf.nn.dropout(fc2, keep_prob=keep_prob)

#     # OUT     Input = [128] Output =  [10]
#     out = tf.matmul(fc1, weights['out']) + biases['out']
    
#     return out

In [4]:
layer_depth = {
    'layer_1': 12,
    'layer_2': 32,
    'layer_3': 32,
    'fully_connected_1': 512,
    'fully_connected_2': 256,
    'out': n_classes,
}

weights = {
    'layer_1': kaiming_he_init((5, 5, 1, layer_depth['layer_1']), 'conv'),
    'layer_2': kaiming_he_init((5, 5, layer_depth['layer_1'], layer_depth['layer_2']), 'conv'),
    'layer_3': kaiming_he_init((3, 3, layer_depth['layer_2'], layer_depth['layer_3']), 'conv'),
    'fully_connected_1': kaiming_he_init((5*5*layer_depth['layer_3'], layer_depth['fully_connected_1'])),
    'fully_connected_2': kaiming_he_init((layer_depth['fully_connected_1'], layer_depth['fully_connected_2'])),
    'out': kaiming_he_init((layer_depth['fully_connected_2'], layer_depth['out']))
}
#+0.3 over the first epochs than +-0

biases = {
    'layer_1': tf.Variable(tf.zeros(layer_depth['layer_1'])),
    'layer_2': tf.Variable(tf.zeros(layer_depth['layer_2'])),
    'layer_3': tf.Variable(tf.zeros(layer_depth['layer_3'])),
    'fully_connected_1': tf.Variable(tf.zeros(layer_depth['fully_connected_1'])),
    'fully_connected_2': tf.Variable(tf.zeros(layer_depth['fully_connected_2'])),
    'out': tf.Variable(tf.zeros(layer_depth['out'])),
}

# Define helper functions
def conv2d(x, W, b, strides=1, padding='VALID'):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding = padding)
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1],
                         strides=[1, k, k, 1],
                         padding='VALID')

keep_prob = tf.placeholder(tf.float32)
def LeNetTransformed(x):
    # Input = 32x32x1. Output = 28x28x12.
    x = tf.expand_dims(x, -1)
    conv1 = conv2d(x, weights['layer_1'], biases['layer_1'])
    # Input = 28x28x12. Output = 14x14x32.
    conv1 = maxpool2d(conv1)
    
    # Output = 10x10x32.
    conv2 = conv2d(conv1, weights['layer_2'], biases['layer_2'])
    
    # Input = 10x10x32. Output = 5x5x32.
#     conv2 = maxpool2d(conv2)
    
    conv3 = conv2d(conv2, weights['layer_3'], biases['layer_3'], padding='SAME')
    conv3 = maxpool2d(conv3)

    # Input = 5x5x32. Output = 320.
    fc0 = tf.contrib.layers.flatten(conv3)
    
    # Input = 320. Output = 512.
    fc1 = tf.add(tf.matmul(fc0, weights['fully_connected_1']), biases['fully_connected_1'])
    fc1 = tf.nn.relu(fc1)
    
    # Add Dropout 
    fc1 = tf.nn.dropout(fc1, keep_prob=keep_prob)
    # +0.02
    
    # Input = 512. Output = 256.
    fc2 = tf.add(tf.matmul(fc1, weights['fully_connected_2']), biases['fully_connected_2'])   
    fc2 = tf.nn.relu(fc2)
    
    # Add Dropout
    fc2 = tf.nn.dropout(fc2, keep_prob=keep_prob)
    # +0.01
    
    # Input = 256. Output = 43.
    logits = tf.add(tf.matmul(fc2, weights['out']), biases['out'])
    
    return logits

# weights = {
#     'c1': tf.Variable(tf.truncated_normal([5, 5, 1, 6], mean=0, stddev=0.1)),
#     'c2': tf.Variable(tf.truncated_normal([5, 5, 6, 16], mean=0, stddev=0.1)),
#     'fc1': tf.Variable(tf.truncated_normal([400, 120], mean=0, stddev=0.1)),
#     'fc2': tf.Variable(tf.truncated_normal([120, 84], mean=0, stddev=0.1)),
#     'out': tf.Variable(tf.truncated_normal([84, 43], mean=0, stddev=0.1))
# }

# biases = {
#     'c1': tf.Variable(tf.truncated_normal([6], mean=0, stddev=0.1)),
#     'c2': tf.Variable(tf.truncated_normal([16], mean=0, stddev=0.1)),
#     'fc1': tf.Variable(tf.truncated_normal([120], mean=0, stddev=0.1)),
#     'fc2': tf.Variable(tf.truncated_normal([84], mean=0, stddev=0.1)),
#     'out': tf.Variable(tf.truncated_normal([43], mean=0, stddev=0.1))
# }

# def conv2d(input_vol, W, b, stride=1, padding='SAME'):
#     conv_layer = tf.nn.conv2d(input_vol, W, strides=[1, stride, stride, 1], padding=padding)
#     conv_layer = tf.nn.bias_add(conv_layer, b)
#     conv_layer = tf.nn.relu(conv_layer)
#     return conv_layer

# def pool2d(input_vol, k=2, stride=2):
#     return tf.nn.max_pool(input_vol, ksize=[1, k, k, 1], strides=[1, stride, stride, 1], padding='SAME')

# def lenet(images, weights, biases, keep_probab):
#     # CONV1 Input = [32x32x1] Output = [28x28x6]
#     images = tf.expand_dims(images, -1)
#     conv1 = conv2d(images, weights['c1'], biases['c1'], padding='VALID')
    
#     # POOL1 Input = [28x28x6] Output = [14x14x6]
#     pool1 = pool2d(conv1)
    
#     # CONV2 Input = [14x14x6] Output = [10x10x16]
#     conv2 = conv2d(pool1, weights['c2'], biases['c2'], padding='VALID')
    
#     # POOL2 Input = [10x10x16] Output = [5x5x16]
#     pool2 = pool2d(conv2)
    
#     # FC1 Input = 400 Output = 120
#     fc1 = tf.reshape(pool2, shape=[-1, weights['fc1'].get_shape().as_list()[0]])
#     fc1 = tf.matmul(fc1, weights['fc1']) + biases['fc1']
#     fc1 = tf.nn.relu(fc1)
    
#     # FC2 Input = 120 Output = 84
#     fc2 = tf.matmul(fc1, weights['fc2']) + biases['fc2']
#     fc2 = tf.nn.relu(fc2)
    
#     # OUT Input = 84 Output = 10
#     out = tf.matmul(fc2, weights['out']) + biases['out']
#     return out

In [5]:
### Training pipeline
logits = LeNetTransformed(X)
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_labels)) +\
#         0.01 * tf.nn.l2_loss(weights['fully_connected_1']) + 0.01 * tf.nn.l2_loss(weights['fully_connected_2'])

# logits = traffic_sign_cnn(X, weights, biases, keep_prob, keep_prob_conv)
# logits = lenet(X, weights, biases, keep_prob)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_labels)) +\
        0.0001 * tf.nn.l2_loss(weights['fully_connected_1']) + 0.0001 * tf.nn.l2_loss(weights['fully_connected_2'])

optimiser = tf.train.AdamOptimizer(learning_rate=learning_rate)\
                .minimize(loss)

In [6]:
### Model Evaluation Pipeline
correct_predictions = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_predictions, dtype=tf.float32))

saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0.0
    session = tf.get_default_session()
    
    for offset in range(0, num_examples, BATCH_SIZE):
        X_batch, y_batch = X_data[offset: offset+BATCH_SIZE], y_data[offset: offset+BATCH_SIZE]
        data_accuracy = session.run(accuracy, feed_dict={
            X: X_batch,
            labels: y_batch,
            keep_prob: 1.0,
            keep_prob_conv: 1.0
        })
        total_accuracy += (data_accuracy * BATCH_SIZE)
    
    return total_accuracy / num_examples

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [7]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.
import time
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    print('Training...')
    num_examples = len(X_train)
          
    for epoch_i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in tnrange(0, num_examples, BATCH_SIZE, desc='Epoch {:>2}'.format(epoch_i + 1)):
            end = offset + BATCH_SIZE
            X_batch, y_batch = X_train[offset:end], y_train[offset:end]
            if epoch_i > 5:                
                LEARNING_RATE = 0.0005
                
            if epoch_i > 10:
                LEARNING_RATE = 0.0001
                
            session.run(optimiser, feed_dict={X: X_batch, labels: y_batch, keep_prob: DROPOUT,
                                             learning_rate: LEARNING_RATE, keep_prob_conv: 0.7})
            
        accuracy_train = evaluate(X_train, y_train)
        accuracy_valid = evaluate(X_valid, y_valid)
        print("EPOCH {} ...".format(epoch_i + 1))
        print("Training Accuracy = {:.5f}".format(accuracy_train))
        print("Validation Accuracy = {:.5f}".format(accuracy_valid))
        print()
    
    ckpt_file = 'ckpts/model-{}-{}-{}'.format(int(accuracy_train*10000), int(accuracy_valid*10000), time.time())
    saver.save(session, ckpt_file)
    print("Model saved at {}".format(ckpt_file))

Training...

EPOCH 1 ...
Training Accuracy = 0.95777
Validation Accuracy = 0.93569


EPOCH 2 ...
Training Accuracy = 0.98706
Validation Accuracy = 0.95909


EPOCH 3 ...
Training Accuracy = 0.99508
Validation Accuracy = 0.97546


EPOCH 4 ...
Training Accuracy = 0.99773
Validation Accuracy = 0.97219


EPOCH 5 ...
Training Accuracy = 0.99710
Validation Accuracy = 0.97006


EPOCH 6 ...
Training Accuracy = 0.99846
Validation Accuracy = 0.98408


EPOCH 7 ...
Training Accuracy = 1.00071
Validation Accuracy = 0.98317


EPOCH 8 ...
Training Accuracy = 1.00053
Validation Accuracy = 0.98213


EPOCH 9 ...
Training Accuracy = 1.00063
Validation Accuracy = 0.98476


EPOCH 10 ...
Training Accuracy = 1.00099
Validation Accuracy = 0.99088

Model saved at ckpts/model-10009-9908-1511998834.1605816


## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")